In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
import json
import base64
import os

# Data loading

In [27]:
df = pd.read_csv(os.path.join('data', 'alzheimers_disease_data.csv'))

In [28]:
df.columns

Index(['PatientID', 'Age', 'Gender', 'Ethnicity', 'EducationLevel', 'BMI',
       'Smoking', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality',
       'SleepQuality', 'FamilyHistoryAlzheimers', 'CardiovascularDisease',
       'Diabetes', 'Depression', 'HeadInjury', 'Hypertension', 'SystolicBP',
       'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL',
       'CholesterolTriglycerides', 'MMSE', 'FunctionalAssessment',
       'MemoryComplaints', 'BehavioralProblems', 'ADL', 'Confusion',
       'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
       'Forgetfulness', 'Diagnosis', 'DoctorInCharge'],
      dtype='object')

## Prediction

In [44]:
def create_tf_example(input):
    feature = {}

    for key, value in input.items():
        if isinstance(value, int):
            feature[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, float):
            feature[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, str):
            feature[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))
    
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

pred = df.drop(['Diagnosis'], axis=1).sample(1).to_dict(orient='records')[0]
# dict_pred = pred
serialized_example = create_tf_example(pred)

In [45]:
json_data = json.dumps({
    "instances": [
        {
            "examples": {
                "b64": base64.b64encode(serialized_example).decode('utf-8')
            }
        }
    ]
})

endpoint = "https://mlops-prediction-production.up.railway.app/v1/models/alzheimer_model:predict"

try:
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    response.raise_for_status()
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.6:
        print('Alzheimers is not detected')
    else:
        print('Alzheimers is detected')
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    print(f"Response content: {response.content}")
except KeyError:
    print("Unexpected response format.")

Alzheimers is not detected


In [46]:
patient = next(iter(pred.items()))[1]
df.Diagnosis[df.PatientID == patient]

192    0
Name: Diagnosis, dtype: int64